In [35]:
import torch
import torch.nn as nn
import numpy as np
from datasets.openml_datasets import OpenML_Dataset

from utils.weakener import Weakener
from models.model import MLP
from utils.losses import PartialLoss,LBLoss,EMLoss,OSLCELoss,OSLBrierLoss
from utils.trainig_testing import train_model,evaluate_model,train_and_evaluate


In [36]:
Data = OpenML_Dataset('iris')
train_x, train_y, test_x, test_y = Data.get_data()

In [37]:
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll')

In [38]:
Weak.M,Weak.Z

(array([[0.   , 0.375, 0.375],
        [0.375, 0.   , 0.375],
        [0.375, 0.375, 0.   ],
        [0.25 , 0.25 , 0.25 ]]),
 array([[0, 1, 1],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]]))

In [39]:
z,w = Weak.generate_weak(train_y)

In [40]:
train_y[:5,:],z[:5],w[:5,:]

(tensor([[0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]),
 tensor([0, 0, 3, 0, 3], dtype=torch.int32),
 tensor([[0., 1., 1.],
         [0., 1., 1.],
         [1., 1., 1.],
         [0., 1., 1.],
         [1., 1., 1.]], dtype=torch.float64))

In [41]:
Weak.generate_wl_priors()

array([0.24074055, 0.24999949, 0.25925996, 0.25      ])

In [42]:
mlp = MLP(Data.num_features,[],Data.num_classes)

In [43]:
mlp

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=3, bias=True)
  )
  (batch_norms): ModuleList()
  (dropout): Dropout(p=0.0, inplace=False)
)

In [44]:
logsoftmax = torch.nn.LogSoftmax(dim=1)
softmax = torch.nn.Softmax(dim=1)

logit  = mlp(train_x)
logp = logsoftmax(logit)
p = softmax(logit)
print(logp[:5],'\n\n',p[:5],'\n\n',torch.exp(logp)[:5])


tensor([[ -2.2520,  -0.1112, -10.1291],
        [ -1.4973,  -0.2536,  -8.2817],
        [ -2.3534,  -0.0999, -11.2415],
        [ -2.0329,  -0.1404, -10.4888],
        [ -0.6985,  -0.7010,  -5.0197]], grad_fn=<SliceBackward0>) 

 tensor([[1.0518e-01, 8.9478e-01, 3.9900e-05],
        [2.2374e-01, 7.7600e-01, 2.5311e-04],
        [9.5047e-02, 9.0494e-01, 1.3119e-05],
        [1.3096e-01, 8.6901e-01, 2.7845e-05],
        [4.9732e-01, 4.9607e-01, 6.6066e-03]], grad_fn=<SliceBackward0>) 

 tensor([[1.0518e-01, 8.9478e-01, 3.9900e-05],
        [2.2374e-01, 7.7600e-01, 2.5311e-04],
        [9.5047e-02, 9.0494e-01, 1.3119e-05],
        [1.3096e-01, 8.6901e-01, 2.7845e-05],
        [4.9732e-01, 4.9607e-01, 6.6066e-03]], grad_fn=<SliceBackward0>)


In [45]:
print(Weak.M,'\n\n',Weak.z[:5],'\n\n',Weak.M[Weak.z][:5,:])

[[0.    0.375 0.375]
 [0.375 0.    0.375]
 [0.375 0.375 0.   ]
 [0.25  0.25  0.25 ]] 

 tensor([0, 0, 3, 0, 3], dtype=torch.int32) 

 [[0.    0.375 0.375]
 [0.    0.375 0.375]
 [0.25  0.25  0.25 ]
 [0.    0.375 0.375]
 [0.25  0.25  0.25 ]]


In [46]:
Q = p[:5,:] * torch.tensor(Weak.M[Weak.z][:5,:])
Q /= torch.sum(Q,dim=1,keepdim=True)

In [47]:
Q

tensor([[0.0000e+00, 9.9996e-01, 4.4590e-05],
        [0.0000e+00, 9.9967e-01, 3.2607e-04],
        [9.5047e-02, 9.0494e-01, 1.3119e-05],
        [0.0000e+00, 9.9997e-01, 3.2041e-05],
        [4.9732e-01, 4.9607e-01, 6.6066e-03]], dtype=torch.float64,
       grad_fn=<DivBackward0>)

In [48]:
class EMLoss(nn.Module):
    def __init__(self,M):
        super(EMLoss, self).__init__()
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.M = M
    def forward(self,out,z):
        logp = logsoftmax(out)
        p = torch.exp(logp)
        Q = p * torch.tensor(self.M[z])
        Q /= torch.sum(Q,dim=1,keepdim=True)
        L = -torch.sum(Q*logp)
        return L


In [49]:
loss = EMLoss(Weak.M)

In [50]:
loss(logit[:5,:],z[:5])

tensor(1.5511, dtype=torch.float64, grad_fn=<NegBackward0>)

In [51]:
[param for param in mlp.parameters()]

[Parameter containing:
 tensor([[ 0.4396,  0.3113,  0.1702,  0.0982],
         [ 0.2022,  0.5303,  0.7224,  0.3857],
         [-0.0492, -0.0053, -0.5295, -0.2086]], requires_grad=True),
 Parameter containing:
 tensor([-0.0263, -0.4498, -0.2378], requires_grad=True)]